In [1]:
import faiss
import os
import pandas as pd
import numpy 
import glob
import numpy as np
from skimage import measure, transform, feature
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
NPZ_FEAT_SRC = "/local/cs572/rchan31/codespace/BaselinePrototype/npz_file_saves/COMPACT_NPZ_DELF_FEATS/"
FAISS_INDEX_FILE = "/local/cs572/rchan31/fINALwEEK/FAISSIndex/FAISS_delf_v1.index" # TODO:
IMG_SRC_PATH = "/local/cs572/cziems/index/"

LOOKUP_CSV = "/local/cs572/rchan31/fINALwEEK/csv_lookups/FAISS_delf_v1_CSV_lookup.csv" # TODO:
N_INDEX = 952133

In [3]:
vecs = []
lookup_by_index = {}
for i, fn in enumerate(sorted(glob.glob(os.path.join(NPZ_FEAT_SRC, "*.npz")))):
    ID = os.path.basename(fn)[:-4]
    vec = np.load(fn)["descriptors"]
    vecs.append(vec[0, :]) # np.asarray
    lookup_by_index[i] = ID
    if i%1000==0:
        print(i, '/ %s' % N_INDEX)
vecs = np.array(vecs)
vecs.shape # (952133, 40)

0 / 952133
1000 / 952133
2000 / 952133
3000 / 952133
4000 / 952133
5000 / 952133
6000 / 952133
7000 / 952133
8000 / 952133
9000 / 952133
10000 / 952133
11000 / 952133
12000 / 952133
13000 / 952133
14000 / 952133
15000 / 952133
16000 / 952133
17000 / 952133
18000 / 952133
19000 / 952133
20000 / 952133
21000 / 952133
22000 / 952133
23000 / 952133
24000 / 952133
25000 / 952133
26000 / 952133
27000 / 952133
28000 / 952133
29000 / 952133
30000 / 952133
31000 / 952133
32000 / 952133
33000 / 952133
34000 / 952133
35000 / 952133
36000 / 952133
37000 / 952133
38000 / 952133
39000 / 952133
40000 / 952133
41000 / 952133
42000 / 952133
43000 / 952133
44000 / 952133
45000 / 952133
46000 / 952133
47000 / 952133
48000 / 952133
49000 / 952133
50000 / 952133
51000 / 952133
52000 / 952133
53000 / 952133
54000 / 952133
55000 / 952133
56000 / 952133
57000 / 952133
58000 / 952133
59000 / 952133
60000 / 952133
61000 / 952133
62000 / 952133
63000 / 952133
64000 / 952133
65000 / 952133
66000 / 952133
67000 / 

(952133, 40)

In [5]:
df = pd.DataFrame(lookup_by_index.values(), columns=["filenames"])
df.to_csv(LOOKUP_CSV)

# Classical Indexing : IndexIVFFlat; quantiser-IndexFlatL2; faiss.METRIC_L2; nlist:n/5

In [8]:
# ============> Attempt-2 : Vector quantized indexing
nlist = int(vecs.shape[0] / 5)
dimension = 40

quantiser = faiss.IndexFlatL2(dimension)  
index = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)

res = faiss.StandardGpuResources()

gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

vecs = np.asarray(vecs, dtype=np.float32)
gpu_index.train(vecs)
gpu_index.add(vecs)

cpu_index = faiss.index_gpu_to_cpu(gpu_index)

print("Indexed vectors : ", cpu_index.ntotal)

faiss.write_index(cpu_index, FAISS_INDEX_FILE)

Indexed vectors :  952133
